# Titanic Dataset Analysis: Machine Learning Predictions

## 1. Project Overview

### Dataset Description

**Overview**

Building upon comprehensive exploratory data analysis, this notebook focuses on developing predictive models to determine passenger survival on the Titanic. 

The data has been split into two groups:
- **training set (train.csv)** - Used to build and validate machine learning models
- **test set (test.csv)** - Used to evaluate model performance on unseen data

The training set provides the outcome **(also known as the "ground truth")** for each passenger. Our models will be based on "features" like passengers' gender, class, and engineered features derived from data analysis insights. 

The test set is used to assess how well our model performs on unseen data. For the test set, we do not provide the ground truth for each passenger. Our objective is to predict these outcomes using the trained models.

We also include **gender_submission.csv**, a baseline prediction assuming all and only female passengers survive, which serves as our benchmark model.

### 🎯 ML Objectives

This machine learning analysis aims to:

1. **Feature Engineering**: Transform raw data into predictive features based on EDA insights
2. **Model Development**: Build and compare multiple ML algorithms (Logistic Regression, Random Forest, SVM, etc.)
3. **Model Validation**: Use cross-validation and proper train/test methodology to avoid overfitting
4. **Performance Optimization**: Tune hyperparameters for optimal predictive accuracy
5. **Model Interpretation**: Understand which features drive survival predictions
6. **Real-world Application**: Create a deployable model for survival prediction

### 📋 Data Dictionary

| Variable | Definition | Key |
|----------|------------|-----|
| **survival** | Survival | 0 = No, 1 = Yes |
| **pclass** | Ticket class | 1 = 1st, 2 = 2nd, 3 = 3rd |
| **sex** | Sex | |
| **age** | Age in years | |
| **sibsp** | # of siblings / spouses aboard the Titanic | |
| **parch** | # of parents / children aboard the Titanic | |
| **ticket** | Ticket number | |
| **fare** | Passenger fare | |
| **cabin** | Cabin number | |
| **embarked** | Port of Embarkation | C = Cherbourg, Q = Queenstown, S = Southampton |

### 📝 Variable Notes

**pclass**: A proxy for socio-economic status (SES)
- 1st = Upper class
- 2nd = Middle class  
- 3rd = Lower class

**age**: Age is fractional if less than 1. If the age is estimated, it is in the form of xx.5

**sibsp**: The dataset defines family relations in this way...
- Sibling = brother, sister, stepbrother, stepsister
- Spouse = husband, wife (mistresses and fiancés were ignored)

**parch**: The dataset defines family relations in this way...
- Parent = mother, father
- Child = daughter, son, stepdaughter, stepson
- Some children travelled only with a nanny, therefore parch=0 for them.

# ================================================================================
# CELL 1: Environment Setup and Library Imports
# ================================================================================

# Core Data Science Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Machine Learning Libraries
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.metrics import precision_score, recall_score, f1_score

# ML Algorithms
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

# Additional Libraries
import sys
from scipy import stats
from scipy.stats import chi2_contingency

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("✅ All libraries imported successfully!")
print(f"Python version: {sys.version}")
print(f"Pandas version: {pd.__version__}")
print(f"Scikit-learn version: {sklearn.__version__}")

# ================================================================================
# CELL 2: Data Loading and Initial Exploration
# ================================================================================

# Load the datasets
print("📊 Loading Titanic datasets...")

# Load training data
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

# Store passenger IDs for final submission
passenger_ids = test_df['PassengerId'].copy()

print(f"✅ Training set loaded: {train_df.shape}")
print(f"✅ Test set loaded: {test_df.shape}")

# Display basic information
print("\n" + "="*50)
print("DATASET OVERVIEW")
print("="*50)

print(f"\nTraining Set Shape: {train_df.shape}")
print(f"Test Set Shape: {test_df.shape}")

print(f"\nTarget Variable Distribution:")
print(train_df['Survived'].value_counts())
print(f"Survival Rate: {train_df['Survived'].mean():.3f}")

print(f"\nTraining Set Info:")
print(train_df.info())

print(f"\nFirst 5 rows of training data:")
print(train_df.head())

# ================================================================================
# CELL 3: Missing Values Analysis
# ================================================================================

print("🔍 MISSING VALUES ANALYSIS")
print("="*50)

# Function to analyze missing values
def analyze_missing_values(df, dataset_name):
    """Comprehensive missing values analysis"""
    missing = df.isnull().sum()
    missing_percent = 100 * missing / len(df)
    
    missing_table = pd.DataFrame({
        'Missing Count': missing,
        'Missing Percentage': missing_percent
    })
    missing_table = missing_table[missing_table['Missing Count'] > 0].sort_values('Missing Count', ascending=False)
    
    print(f"\n{dataset_name} Missing Values:")
    print(missing_table)
    
    return missing_table

# Analyze missing values in both datasets
train_missing = analyze_missing_values(train_df, "TRAINING SET")
test_missing = analyze_missing_values(test_df, "TEST SET")

# Visualize missing values
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Training set missing values
train_missing_viz = train_df.isnull().sum()
train_missing_viz = train_missing_viz[train_missing_viz > 0].sort_values(ascending=False)
axes[0].bar(train_missing_viz.index, train_missing_viz.values, color='coral')
axes[0].set_title('Missing Values - Training Set')
axes[0].set_ylabel('Count')
axes[0].tick_params(axis='x', rotation=45)

# Test set missing values
test_missing_viz = test_df.isnull().sum()
test_missing_viz = test_missing_viz[test_missing_viz > 0].sort_values(ascending=False)
axes[1].bar(test_missing_viz.index, test_missing_viz.values, color='lightblue')
axes[1].set_title('Missing Values - Test Set')
axes[1].set_ylabel('Count')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# ================================================================================
# CELL 4: Feature Engineering Functions
# ================================================================================

print("🔧 FEATURE ENGINEERING SETUP")
print("="*50)

def extract_title(name):
    """Extract title from passenger name"""
    title = name.split(',')[1].split('.')[0].strip()
    # Group rare titles
    title_mapping = {
        'Mr': 'Mr',
        'Miss': 'Miss', 
        'Mrs': 'Mrs',
        'Master': 'Master',
        'Dr': 'Rare',
        'Rev': 'Rare',
        'Major': 'Rare',
        'Col': 'Rare',
        'Capt': 'Rare',
        'Countess': 'Rare',
        'Don': 'Rare',
        'Dona': 'Rare',
        'Jonkheer': 'Rare',
        'Lady': 'Rare',
        'Mlle': 'Miss',
        'Mme': 'Mrs',
        'Ms': 'Miss',
        'Sir': 'Rare'
    }
    return title_mapping.get(title, 'Rare')

def create_family_features(df):
    """Create family-related features"""
    df = df.copy()
    
    # Family size
    df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
    
    # Family size categories
    df['FamilySizeGroup'] = 'Medium'
    df.loc[df['FamilySize'] == 1, 'FamilySizeGroup'] = 'Alone'
    df.loc[df['FamilySize'] >= 5, 'FamilySizeGroup'] = 'Large'
    
    # Is alone
    df['IsAlone'] = (df['FamilySize'] == 1).astype(int)
    
    return df

def create_age_features(df):
    """Create age-related features"""
    df = df.copy()
    
    # Age groups
    df['AgeGroup'] = 'Adult'
    df.loc[df['Age'] <= 16, 'AgeGroup'] = 'Child'
    df.loc[df['Age'] >= 60, 'AgeGroup'] = 'Senior'
    
    # Is child
    df['IsChild'] = (df['Age'] <= 16).astype(int)
    
    return df

def create_fare_features(df):
    """Create fare-related features"""
    df = df.copy()
    
    # Fare per person
    df['FarePerPerson'] = df['Fare'] / df['FamilySize']
    
    # Fare categories based on quantiles
    fare_bins = [0, 7.91, 14.454, 31, np.inf]
    fare_labels = ['Low', 'Medium', 'High', 'Very High']
    df['FareGroup'] = pd.cut(df['Fare'], bins=fare_bins, labels=fare_labels, include_lowest=True)
    
    return df

def create_cabin_features(df):
    """Create cabin-related features"""
    df = df.copy()
    
    # Has cabin
    df['HasCabin'] = df['Cabin'].notna().astype(int)
    
    # Cabin deck (first letter)
    df['CabinDeck'] = df['Cabin'].str[0]
    df['CabinDeck'] = df['CabinDeck'].fillna('Unknown')
    
    return df

def create_ticket_features(df):
    """Create ticket-related features"""
    df = df.copy()
    
    # Ticket length
    df['TicketLength'] = df['Ticket'].str.len()
    
    # Has numeric ticket
    df['HasNumericTicket'] = df['Ticket'].str.isdigit().astype(int)
    
    return df

print("✅ Feature engineering functions defined!")

# ================================================================================
# CELL 5: Data Preprocessing Pipeline
# ================================================================================

def preprocess_data(df, is_training=True):
    """
    Comprehensive data preprocessing pipeline
    """
    print(f"\n🔄 Preprocessing {'training' if is_training else 'test'} data...")
    
    df = df.copy()
    
    # 1. Extract titles
    df['Title'] = df['Name'].apply(extract_title)
    
    # 2. Create family features
    df = create_family_features(df)
    
    # 3. Handle missing Ages using median by Title and Pclass
    age_median = df.groupby(['Title', 'Pclass'])['Age'].median()
    for title in df['Title'].unique():
        for pclass in df['Pclass'].unique():
            mask = (df['Title'] == title) & (df['Pclass'] == pclass) & df['Age'].isna()
            if mask.sum() > 0:
                median_age = age_median.get((title, pclass))
                if pd.notna(median_age):
                    df.loc[mask, 'Age'] = median_age
                else:
                    # Fallback to overall median for title
                    overall_median = df[df['Title'] == title]['Age'].median()
                    if pd.notna(overall_median):
                        df.loc[mask, 'Age'] = overall_median
                    else:
                        df.loc[mask, 'Age'] = df['Age'].median()
    
    # 4. Create age features
    df = create_age_features(df)
    
    # 5. Handle missing Embarked (mode)
    df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])
    
    # 6. Handle missing Fare (median by Pclass)
    if df['Fare'].isna().sum() > 0:
        fare_median = df.groupby('Pclass')['Fare'].median()
        for pclass in df['Pclass'].unique():
            mask = (df['Pclass'] == pclass) & df['Fare'].isna()
            if mask.sum() > 0:
                df.loc[mask, 'Fare'] = fare_median[pclass]
    
    # 7. Create fare features
    df = create_fare_features(df)
    
    # 8. Create cabin features
    df = create_cabin_features(df)
    
    # 9. Create ticket features
    df = create_ticket_features(df)
    
    # 10. Drop original columns that are no longer needed
    columns_to_drop = ['Name', 'Ticket', 'Cabin']
    df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])
    
    print(f"✅ Preprocessing complete. Final shape: {df.shape}")
    
    return df

# Apply preprocessing to both datasets
print("🔄 APPLYING PREPROCESSING PIPELINE")
print("="*50)

train_processed = preprocess_data(train_df, is_training=True)
test_processed = preprocess_data(test_df, is_training=False)

print(f"\nProcessed Training Set Shape: {train_processed.shape}")
print(f"Processed Test Set Shape: {test_processed.shape}")

# Check for remaining missing values
print(f"\nRemaining missing values in training set:")
print(train_processed.isnull().sum().sum())

print(f"\nRemaining missing values in test set:")
print(test_processed.isnull().sum().sum())

# Display processed data sample
print(f"\nProcessed training data sample:")
print(train_processed.head())

# ================================================================================
# CELL 6: Feature Analysis and Selection
# ================================================================================

print("📊 FEATURE ANALYSIS AND SELECTION")
print("="*50)

# Separate features and target
X_train_processed = train_processed.drop(['Survived', 'PassengerId'], axis=1)
y_train = train_processed['Survived']

print(f"Features shape: {X_train_processed.shape}")
print(f"Target shape: {y_train.shape}")

# Analyze categorical vs numerical features
categorical_features = X_train_processed.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_features = X_train_processed.select_dtypes(include=['int64', 'float64']).columns.tolist()

print(f"\nCategorical features ({len(categorical_features)}): {categorical_features}")
print(f"Numerical features ({len(numerical_features)}): {numerical_features}")

# Correlation analysis for numerical features
plt.figure(figsize=(12, 8))
correlation_matrix = X_train_processed[numerical_features].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, fmt='.2f')
plt.title('Correlation Matrix - Numerical Features')
plt.tight_layout()
plt.show()

# Feature importance using mutual information
from sklearn.feature_selection import mutual_info_classif

# Encode categorical variables for mutual information
X_encoded = X_train_processed.copy()
le_dict = {}

for col in categorical_features:
    le = LabelEncoder()
    X_encoded[col] = le.fit_transform(X_encoded[col].astype(str))
    le_dict[col] = le

# Calculate mutual information
mi_scores = mutual_info_classif(X_encoded, y_train)
mi_scores = pd.Series(mi_scores, index=X_encoded.columns).sort_values(ascending=False)

# Plot feature importance
plt.figure(figsize=(12, 8))
mi_scores.plot(kind='barh', color='skyblue')
plt.title('Feature Importance (Mutual Information)')
plt.xlabel('Mutual Information Score')
plt.tight_layout()
plt.show()

print("Top 10 most important features:")
print(mi_scores.head(10))

# ================================================================================
# CELL 7: Data Encoding and Scaling
# ================================================================================

print("🔢 DATA ENCODING AND SCALING")
print("="*50)

def encode_features(X_train, X_test, categorical_features):
    """
    Encode categorical features using one-hot encoding
    """
    X_train_encoded = X_train.copy()
    X_test_encoded = X_test.copy()
    
    # One-hot encode categorical features
    for feature in categorical_features:
        # Get all unique values from both train and test
        all_categories = list(set(X_train[feature].unique()) | set(X_test[feature].unique()))
        
        # Create dummy variables
        train_dummies = pd.get_dummies(X_train[feature], prefix=feature)
        test_dummies = pd.get_dummies(X_test[feature], prefix=feature)
        
        # Ensure both have same columns
        for category in all_categories:
            col_name = f"{feature}_{category}"
            if col_name not in train_dummies.columns:
                train_dummies[col_name] = 0
            if col_name not in test_dummies.columns:
                test_dummies[col_name] = 0
        
        # Sort columns to ensure same order
        train_dummies = train_dummies.reindex(sorted(train_dummies.columns), axis=1)
        test_dummies = test_dummies.reindex(sorted(test_dummies.columns), axis=1)
        
        # Drop original feature and add dummies
        X_train_encoded = X_train_encoded.drop(feature, axis=1)
        X_test_encoded = X_test_encoded.drop(feature, axis=1)
        
        X_train_encoded = pd.concat([X_train_encoded, train_dummies], axis=1)
        X_test_encoded = pd.concat([X_test_encoded, test_dummies], axis=1)
    
    return X_train_encoded, X_test_encoded

# Prepare test features (remove PassengerId)
X_test_processed = test_processed.drop('PassengerId', axis=1)

# Encode features
X_train_encoded, X_test_encoded = encode_features(
    X_train_processed, X_test_processed, categorical_features
)

print(f"Encoded training features shape: {X_train_encoded.shape}")
print(f"Encoded test features shape: {X_test_encoded.shape}")

# Scale numerical features
scaler = StandardScaler()

# Identify numerical columns in encoded data
numerical_cols_encoded = X_train_encoded.select_dtypes(include=['int64', 'float64']).columns
categorical_cols_encoded = [col for col in X_train_encoded.columns if col not in numerical_cols_encoded]

print(f"Numerical columns to scale: {len(numerical_cols_encoded)}")
print(f"Categorical (dummy) columns: {len(categorical_cols_encoded)}")

# Scale only numerical features
X_train_scaled = X_train_encoded.copy()
X_test_scaled = X_test_encoded.copy()

if len(numerical_cols_encoded) > 0:
    X_train_scaled[numerical_cols_encoded] = scaler.fit_transform(X_train_encoded[numerical_cols_encoded])
    X_test_scaled[numerical_cols_encoded] = scaler.transform(X_test_encoded[numerical_cols_encoded])

print(f"✅ Feature encoding and scaling complete!")
print(f"Final training features shape: {X_train_scaled.shape}")
print(f"Final test features shape: {X_test_scaled.shape}")

# ================================================================================
# CELL 8: Train-Validation Split
# ================================================================================

print("🔄 CREATING TRAIN-VALIDATION SPLIT")
print("="*50)

# Split the training data into train and validation sets
X_train_final, X_val_final, y_train_final, y_val_final = train_test_split(
    X_train_scaled, y_train, 
    test_size=0.2, 
    random_state=42, 
    stratify=y_train
)

print(f"Training set: {X_train_final.shape}")
print(f"Validation set: {X_val_final.shape}")
print(f"Training target distribution:")
print(f"  Survived: {y_train_final.sum()} ({y_train_final.mean():.3f})")
print(f"  Not Survived: {len(y_train_final) - y_train_final.sum()} ({1 - y_train_final.mean():.3f})")

print(f"Validation target distribution:")
print(f"  Survived: {y_val_final.sum()} ({y_val_final.mean():.3f})")
print(f"  Not Survived: {len(y_val_final) - y_val_final.sum()} ({1 - y_val_final.mean():.3f})")

# ================================================================================
# CELL 9: Model Definition and Training
# ================================================================================

print("🤖 MODEL DEFINITION AND TRAINING")
print("="*50)

# Define models to test
models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'Random Forest': RandomForestClassifier(random_state=42, n_estimators=100),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'SVM': SVC(random_state=42, probability=True),
    'KNN': KNeighborsClassifier(),
    'Naive Bayes': GaussianNB(),
    'Decision Tree': DecisionTreeClassifier(random_state=42)
}

# Function to evaluate models
def evaluate_model(model, X_train, X_val, y_train, y_val, model_name):
    """
    Train and evaluate a model
    """
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions
    y_train_pred = model.predict(X_train)
    y_val_pred = model.predict(X_val)
    
    # Calculate probabilities for ROC-AUC
    try:
        y_val_proba = model.predict_proba(X_val)[:, 1]
        roc_auc = roc_auc_score(y_val, y_val_proba)
    except:
        roc_auc = None
    
    # Calculate metrics
    train_accuracy = accuracy_score(y_train, y_train_pred)
    val_accuracy = accuracy_score(y_val, y_val_pred)
    val_precision = precision_score(y_val, y_val_pred)
    val_recall = recall_score(y_val, y_val_pred)
    val_f1 = f1_score(y_val, y_val_pred)
    
    results = {
        'Model': model_name,
        'Train Accuracy': train_accuracy,
        'Val Accuracy': val_accuracy,
        'Precision': val_precision,
        'Recall': val_recall,
        'F1 Score': val_f1,
        'ROC-AUC': roc_auc
    }
    
    return results, model

# Train and evaluate all models
results_list = []
trained_models = {}

print("Training and evaluating models...")

for name, model in models.items():
    print(f"\n🔄 Training {name}...")
    
    results, trained_model = evaluate_model(
        model, X_train_final, X_val_final, y_train_final, y_val_final, name
    )
    
    results_list.append(results)
    trained_models[name] = trained_model
    
    print(f"✅ {name} - Val Accuracy: {results['Val Accuracy']:.4f}")

# Create results DataFrame
results_df = pd.DataFrame(results_list)
results_df = results_df.sort_values('Val Accuracy', ascending=False)

print(f"\n📊 MODEL COMPARISON RESULTS")
print("="*50)
print(results_df.round(4))

# ================================================================================
# CELL 10: Cross-Validation Analysis
# ================================================================================

print("🔍 CROSS-VALIDATION ANALYSIS")
print("="*50)

# Perform cross-validation for more robust evaluation
cv_scores = {}
cv_folds = 5

print(f"Performing {cv_folds}-fold cross-validation...")

for name, model in models.items():
    print(f"\n🔄 CV for {name}...")
    
    # Perform cross-validation
    cv_score = cross_val_score(
        model, X_train_scaled, y_train, 
        cv=StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42),
        scoring='accuracy'
    )
    
    cv_scores[name] = {
        'Mean CV Score': cv_score.mean(),
        'Std CV Score': cv_score.std(),
        'CV Scores': cv_score
    }
    
    print(f"✅ {name} - CV Score: {cv_score.mean():.4f} (+/- {cv_score.std() * 2:.4f})")

# Create CV results DataFrame
cv_results = []
for name, scores in cv_scores.items():
    cv_results.append({
        'Model': name,
        'Mean CV Score': scores['Mean CV Score'],
        'Std CV Score': scores['Std CV Score']
    })

cv_results_df = pd.DataFrame(cv_results)
cv_results_df = cv_results_df.sort_values('Mean CV Score', ascending=False)

print(f"\n📊 CROSS-VALIDATION RESULTS")
print("="*50)
print(cv_results_df.round(4))

# Visualize CV results
plt.figure(figsize=(12, 6))
plt.errorbar(cv_results_df['Model'], cv_results_df['Mean CV Score'], 
             yerr=cv_results_df['Std CV Score'], fmt='o', capsize=5)
plt.xticks(rotation=45)
plt.ylabel('Cross-Validation Accuracy')
plt.title('Model Performance with Cross-Validation')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# ================================================================================
# CELL 11: Hyperparameter Tuning
# ================================================================================

print("🎛️ HYPERPARAMETER TUNING")
print("="*50)

# Select top 3 models for hyperparameter tuning
top_models = cv_results_df.head(3)['Model'].tolist()
print(f"Tuning hyperparameters for top models: {top_models}")

# Define parameter grids
param_grids = {
    'Random Forest': {
        'n_estimators': [100, 200, 300],
        'max_depth': [5, 10, 15, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    },
    'Gradient Boosting': {
        'n_estimators': [100, 200, 300],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 5, 7],
        'min_samples_split': [2, 5, 10]
    },
    'Logistic Regression': {
        'C': [0.1, 1, 10, 100],
        'penalty': ['l1', 'l2'],
        'solver': ['liblinear', 'saga']
    },
    'SVM': {
        'C': [0.1, 1, 10],
        'kernel': ['rbf', 'linear'],
        'gamma': ['scale', 'auto']
    }
}

# Perform hyperparameter tuning
tuned_models = {}
tuning_results = []

for model_name in top_models:
    if model_name in param_grids:
        print(f"✅ {model_name} - Best CV Score: {grid_search.best_score_:.4f}")
        print(f"   Validation Accuracy: {tuned_accuracy:.4f}")
        print(f"   Best Parameters: {grid_search.best_params_}")

# Display tuning results
print(f"\n📊 HYPERPARAMETER TUNING RESULTS")
print("="*50)
for result in tuning_results:
    print(f"\n{result['Model']}:")
    print(f"  Best CV Score: {result['Best Score (CV)']:.4f}")
    print(f"  Validation Accuracy: {result['Validation Accuracy']:.4f}")
    print(f"  Best Parameters: {result['Best Parameters']}")

# ================================================================================
# CELL 12: Model Ensemble and Final Model Selection
# ================================================================================

print("🏆 MODEL ENSEMBLE AND FINAL SELECTION")
print("="*50)

# Create ensemble model with top performers
ensemble_models = []
ensemble_names = []

# Select best tuned models
for result in tuning_results:
    model_name = result['Model']
    if model_name in tuned_models:
        ensemble_models.append((model_name.lower().replace(' ', '_'), tuned_models[model_name]))
        ensemble_names.append(model_name)

print(f"Creating ensemble with: {ensemble_names}")

# Create voting classifier
voting_clf = VotingClassifier(
    estimators=ensemble_models,
    voting='soft'  # Use probability-based voting
)

# Train ensemble
print("\n🔄 Training ensemble model...")
voting_clf.fit(X_train_final, y_train_final)

# Evaluate ensemble
y_val_pred_ensemble = voting_clf.predict(X_val_final)
ensemble_accuracy = accuracy_score(y_val_final, y_val_pred_ensemble)
ensemble_precision = precision_score(y_val_final, y_val_pred_ensemble)
ensemble_recall = recall_score(y_val_final, y_val_pred_ensemble)
ensemble_f1 = f1_score(y_val_final, y_val_pred_ensemble)

print(f"✅ Ensemble Model Performance:")
print(f"   Validation Accuracy: {ensemble_accuracy:.4f}")
print(f"   Precision: {ensemble_precision:.4f}")
print(f"   Recall: {ensemble_recall:.4f}")
print(f"   F1 Score: {ensemble_f1:.4f}")

# Compare with individual models
print(f"\n📊 FINAL MODEL COMPARISON")
print("="*50)

final_comparison = []

# Add individual tuned models
for result in tuning_results:
    final_comparison.append({
        'Model': result['Model'] + ' (Tuned)',
        'Validation Accuracy': result['Validation Accuracy'],
        'Type': 'Individual'
    })

# Add ensemble
final_comparison.append({
    'Model': 'Ensemble (Voting)',
    'Validation Accuracy': ensemble_accuracy,
    'Type': 'Ensemble'
})

final_df = pd.DataFrame(final_comparison)
final_df = final_df.sort_values('Validation Accuracy', ascending=False)
print(final_df)

# Select best model
best_model_row = final_df.iloc[0]
best_model_name = best_model_row['Model']
best_accuracy = best_model_row['Validation Accuracy']

print(f"\n🏆 BEST MODEL: {best_model_name}")
print(f"   Validation Accuracy: {best_accuracy:.4f}")

# ================================================================================
# CELL 13: Feature Importance Analysis
# ================================================================================

print("📊 FEATURE IMPORTANCE ANALYSIS")
print("="*50)

# Get feature importance from the best performing individual model
# Use Random Forest for feature importance (most interpretable)
if 'Random Forest' in tuned_models:
    rf_model = tuned_models['Random Forest']
elif 'Random Forest' in trained_models:
    rf_model = trained_models['Random Forest']
else:
    # Train a new Random Forest for feature importance
    rf_model = RandomForestClassifier(random_state=42, n_estimators=100)
    rf_model.fit(X_train_final, y_train_final)

# Get feature importance
feature_importance = pd.DataFrame({
    'Feature': X_train_final.columns,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=False)

print("Top 15 Most Important Features:")
print(feature_importance.head(15))

# Plot feature importance
plt.figure(figsize=(12, 8))
top_features = feature_importance.head(15)
plt.barh(range(len(top_features)), top_features['Importance'], color='skyblue')
plt.yticks(range(len(top_features)), top_features['Feature'])
plt.xlabel('Feature Importance')
plt.title('Top 15 Feature Importance (Random Forest)')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

# Analyze feature groups
feature_groups = {
    'Personal': ['Sex', 'Age', 'Title'],
    'Family': ['FamilySize', 'IsAlone', 'SibSp', 'Parch'],
    'Economic': ['Pclass', 'Fare', 'FareGroup', 'FarePerPerson'],
    'Location': ['Embarked', 'CabinDeck', 'HasCabin'],
    'Other': ['TicketLength', 'HasNumericTicket']
}

group_importance = {}
for group, features in feature_groups.items():
    group_features = [f for f in features if any(f in col for col in feature_importance['Feature'])]
    if group_features:
        # Sum importance of features containing group feature names
        importance_sum = 0
        for feature in group_features:
            matching_cols = [col for col in feature_importance['Feature'] if feature in col]
            importance_sum += feature_importance[feature_importance['Feature'].isin(matching_cols)]['Importance'].sum()
        group_importance[group] = importance_sum

# Plot group importance
plt.figure(figsize=(10, 6))
groups = list(group_importance.keys())
importances = list(group_importance.values())
plt.bar(groups, importances, color='lightcoral')
plt.xlabel('Feature Groups')
plt.ylabel('Total Importance')
plt.title('Feature Importance by Groups')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print("\nFeature Group Importance:")
for group, importance in sorted(group_importance.items(), key=lambda x: x[1], reverse=True):
    print(f"  {group}: {importance:.4f}")

# ================================================================================
# CELL 14: Model Evaluation and Confusion Matrix
# ================================================================================

print("📈 DETAILED MODEL EVALUATION")
print("="*50)

# Select final model for evaluation
if best_model_name == 'Ensemble (Voting)':
    final_model = voting_clf
    model_for_eval = voting_clf
else:
    # Find the corresponding tuned model
    model_key = best_model_name.replace(' (Tuned)', '')
    final_model = tuned_models[model_key]
    model_for_eval = final_model

# Make predictions
y_val_pred_final = model_for_eval.predict(X_val_final)
y_val_proba_final = model_for_eval.predict_proba(X_val_final)[:, 1]

# Calculate all metrics
final_accuracy = accuracy_score(y_val_final, y_val_pred_final)
final_precision = precision_score(y_val_final, y_val_pred_final)
final_recall = recall_score(y_val_final, y_val_pred_final)
final_f1 = f1_score(y_val_final, y_val_pred_final)
final_roc_auc = roc_auc_score(y_val_final, y_val_proba_final)

print(f"FINAL MODEL PERFORMANCE ({best_model_name}):")
print(f"  Accuracy:  {final_accuracy:.4f}")
print(f"  Precision: {final_precision:.4f}")
print(f"  Recall:    {final_recall:.4f}")
print(f"  F1 Score:  {final_f1:.4f}")
print(f"  ROC-AUC:   {final_roc_auc:.4f}")

# Confusion Matrix
cm = confusion_matrix(y_val_final, y_val_pred_final)

plt.figure(figsize=(15, 5))

# Plot 1: Confusion Matrix
plt.subplot(1, 3, 1)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Not Survived', 'Survived'],
            yticklabels=['Not Survived', 'Survived'])
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')

# Plot 2: ROC Curve
plt.subplot(1, 3, 2)
fpr, tpr, _ = roc_curve(y_val_final, y_val_proba_final)
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (AUC = {final_roc_auc:.3f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")

# Plot 3: Prediction Distribution
plt.subplot(1, 3, 3)
plt.hist(y_val_proba_final[y_val_final == 0], bins=20, alpha=0.7, label='Not Survived', color='red')
plt.hist(y_val_proba_final[y_val_final == 1], bins=20, alpha=0.7, label='Survived', color='blue')
plt.xlabel('Predicted Probability')
plt.ylabel('Frequency')
plt.title('Prediction Probability Distribution')
plt.legend()

plt.tight_layout()
plt.show()

# Classification Report
print(f"\nCLASSIFICATION REPORT:")
print("="*40)
print(classification_report(y_val_final, y_val_pred_final, 
                          target_names=['Not Survived', 'Survived']))

# ================================================================================
# CELL 15: Cross-Validation on Final Model
# ================================================================================

print("🔄 FINAL MODEL CROSS-VALIDATION")
print("="*50)

# Perform comprehensive cross-validation on final model
cv_scores_final = cross_val_score(
    model_for_eval, X_train_scaled, y_train, 
    cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=42),
    scoring='accuracy'
)

print(f"10-Fold Cross-Validation Results for {best_model_name}:")
print(f"  Mean Accuracy: {cv_scores_final.mean():.4f}")
print(f"  Std Deviation: {cv_scores_final.std():.4f}")
print(f"  95% Confidence Interval: {cv_scores_final.mean():.4f} ± {1.96 * cv_scores_final.std():.4f}")

# Plot CV scores
plt.figure(figsize=(10, 6))
plt.boxplot(cv_scores_final, labels=[best_model_name])
plt.ylabel('Accuracy')
plt.title('Cross-Validation Score Distribution')
plt.grid(True, alpha=0.3)
plt.show()

print(f"\nIndividual CV Scores:")
for i, score in enumerate(cv_scores_final, 1):
    print(f"  Fold {i:2d}: {score:.4f}")

# ================================================================================
# CELL 16: Learning Curves
# ================================================================================

print("📈 LEARNING CURVES ANALYSIS")
print("="*50)

from sklearn.model_selection import learning_curve

# Generate learning curves
train_sizes, train_scores, val_scores = learning_curve(
    model_for_eval, X_train_scaled, y_train,
    cv=5, n_jobs=-1, 
    train_sizes=np.linspace(0.1, 1.0, 10),
    random_state=42
)

# Calculate mean and std
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)
val_mean = np.mean(val_scores, axis=1)
val_std = np.std(val_scores, axis=1)

# Plot learning curves
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(train_sizes, train_mean, 'o-', color='blue', label='Training Score')
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.1, color='blue')
plt.plot(train_sizes, val_mean, 'o-', color='red', label='Validation Score')
plt.fill_between(train_sizes, val_mean - val_std, val_mean + val_std, alpha=0.1, color='red')
plt.xlabel('Training Set Size')
plt.ylabel('Accuracy')
plt.title('Learning Curves')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot bias-variance analysis
plt.subplot(1, 2, 2)
bias = train_mean - val_mean
plt.plot(train_sizes, bias, 'o-', color='green', label='Bias (Train - Val)')
plt.axhline(y=0, color='black', linestyle='--', alpha=0.5)
plt.xlabel('Training Set Size')
plt.ylabel('Bias')
plt.title('Bias Analysis')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Analyze results
final_bias = bias[-1]
final_variance = val_std[-1]

print(f"Learning Curve Analysis:")
print(f"  Final Training Score: {train_mean[-1]:.4f} ± {train_std[-1]:.4f}")
print(f"  Final Validation Score: {val_mean[-1]:.4f} ± {val_std[-1]:.4f}")
print(f"  Bias (Underfitting): {final_bias:.4f}")
print(f"  Variance (Overfitting): {final_variance:.4f}")

if final_bias > 0.05:
    print("  ⚠️  Model may be underfitting (high bias)")
elif final_bias < -0.05:
    print("  ⚠️  Model may be overfitting (negative bias)")
else:
    print("  ✅ Model bias is acceptable")

if final_variance > 0.05:
    print("  ⚠️  Model has high variance")
else:
    print("  ✅ Model variance is acceptable")

# ================================================================================
# CELL 17: Make Predictions on Test Set
# ================================================================================

print("🎯 MAKING PREDICTIONS ON TEST SET")
print("="*50)

# Train final model on entire training set
print(f"Training final model on entire training set...")
model_for_eval.fit(X_train_scaled, y_train)

# Make predictions on test set
print(f"Making predictions on test set...")
test_predictions = model_for_eval.predict(X_test_scaled)
test_probabilities = model_for_eval.predict_proba(X_test_scaled)[:, 1]

print(f"✅ Predictions completed!")
print(f"Test set size: {len(test_predictions)}")
print(f"Predicted survival rate: {test_predictions.mean():.3f}")

# Analyze predictions
prediction_counts = pd.Series(test_predictions).value_counts().sort_index()
print(f"\nPrediction Distribution:")
print(f"  Not Survived (0): {prediction_counts[0]} ({prediction_counts[0]/len(test_predictions)*100:.1f}%)")
print(f"  Survived (1): {prediction_counts[1]} ({prediction_counts[1]/len(test_predictions)*100:.1f}%)")

# Plot prediction distribution
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.bar(['Not Survived', 'Survived'], prediction_counts.values, color=['red', 'blue'], alpha=0.7)
plt.ylabel('Count')
plt.title('Test Set Predictions Distribution')
for i, v in enumerate(prediction_counts.values):
    plt.text(i, v + 1, str(v), ha='center', va='bottom')

plt.subplot(1, 2, 2)
plt.hist(test_probabilities, bins=20, alpha=0.7, color='green', edgecolor='black')
plt.xlabel('Predicted Probability of Survival')
plt.ylabel('Frequency')
plt.title('Prediction Probability Distribution')
plt.axvline(x=0.5, color='red', linestyle='--', label='Decision Threshold')
plt.legend()

plt.tight_layout()
plt.show()

# ================================================================================
# CELL 18: Create Submission File
# ================================================================================

print("📄 CREATING SUBMISSION FILE")
print("="*50)

# Create submission DataFrame
submission = pd.DataFrame({
    'PassengerId': passenger_ids,
    'Survived': test_predictions
})

# Save submission file
submission_filename = 'titanic_predictions.csv'
submission.to_csv(submission_filename, index=False)

print(f"✅ Submission file created: {submission_filename}")
print(f"Submission shape: {submission.shape}")
print(f"\nFirst 10 predictions:")
print(submission.head(10))

# Create detailed prediction file with probabilities
detailed_submission = pd.DataFrame({
    'PassengerId': passenger_ids,
    'Survived': test_predictions,
    'Survival_Probability': test_probabilities
})

detailed_filename = 'titanic_detailed_predictions.csv'
detailed_submission.to_csv(detailed_filename, index=False)

print(f"✅ Detailed submission file created: {detailed_filename}")

# Validation checks
print(f"\n🔍 SUBMISSION VALIDATION:")
print(f"  PassengerId range: {submission['PassengerId'].min()} - {submission['PassengerId'].max()}")
print(f"  Unique PassengerIds: {submission['PassengerId'].nunique()}")
print(f"  Predictions are binary: {set(submission['Survived'].unique()) == {0, 1}}")
print(f"  No missing values: {submission.isnull().sum().sum() == 0}")

# ================================================================================
# CELL 19: Model Interpretation and Insights
# ================================================================================

print("🧠 MODEL INTERPRETATION AND INSIGHTS")
print("="*50)

# Key insights from the analysis
print("KEY INSIGHTS FROM TITANIC SURVIVAL ANALYSIS:")
print("="*45)

print("\n1. 📊 FEATURE IMPORTANCE INSIGHTS:")
top_5_features = feature_importance.head(5)
for idx, row in top_5_features.iterrows():
    print(f"   • {row['Feature']}: {row['Importance']:.4f}")

print(f"\n2. 🎯 MODEL PERFORMANCE:")
print(f"   • Best Model: {best_model_name}")
print(f"   • Cross-Validation Accuracy: {cv_scores_final.mean():.4f} ± {cv_scores_final.std():.4f}")
print(f"   • Validation Accuracy: {final_accuracy:.4f}")
print(f"   • ROC-AUC Score: {final_roc_auc:.4f}")

print(f"\n3. 📈 SURVIVAL PREDICTIONS:")
print(f"   • Training Set Survival Rate: {y_train.mean():.3f}")
print(f"   • Test Set Predicted Survival Rate: {test_predictions.mean():.3f}")
print(f"   • Model captures realistic survival patterns")

print(f"\n4. 🔍 KEY SURVIVAL FACTORS:")
print(f"   • Gender appears to be the strongest predictor")
print(f"   • Passenger class (socioeconomic status) is crucial")
print(f"   • Age and family composition matter significantly")
print(f"   • Fare and cabin location provide additional predictive power")

print(f"\n5. ⚖️ MODEL RELIABILITY:")
if final_bias < 0.05 and final_variance < 0.05:
    print(f"   • Model shows good bias-variance balance")
else:
    print(f"   • Model may need further tuning for optimal performance")

print(f"   • Cross-validation shows consistent performance")
print(f"   • Feature importance aligns with historical context")

# ================================================================================
# CELL 20: Conclusion and Next Steps
# ================================================================================

print("🎉 PROJECT CONCLUSION")
print("="*50)

print("MACHINE LEARNING PIPELINE COMPLETED SUCCESSFULLY!")
print("\n📋 DELIVERABLES CREATED:")
print(f"   ✅ Trained and validated ML models")
print(f"   ✅ Feature importance analysis")
print(f"   ✅ Model performance evaluation")
print(f"   ✅ Test set predictions: {submission_filename}")
print(f"   ✅ Detailed predictions: {detailed_filename}")

print(f"\n🏆 FINAL MODEL SUMMARY:")
print(f"   Model Type: {best_model_name}")
print(f"   Validation Accuracy: {final_accuracy:.4f}")
print(f"   Cross-Validation: {cv_scores_final.mean():.4f} ± {cv_scores_final.std():.4f}")
print(f"   ROC-AUC: {final_roc_auc:.4f}")

print(f"\n🚀 POTENTIAL IMPROVEMENTS:")
print(f"   • Feature engineering: Extract more information from names, tickets")
print(f"   • Advanced ensemble methods: Stacking, blending")
print(f"   • Deep learning approaches for complex pattern detection")
print(f"   • External data integration: Historical context, ship layout")
print(f"   • Hyperparameter optimization with Bayesian methods")

print(f"\n💼 BUSINESS VALUE:")
print(f"   • Demonstrates end-to-end ML pipeline capabilities")
print(f"   • Shows proper model validation and evaluation practices")
print(f"   • Provides interpretable insights for decision-making")
print(f"   • Achieves competitive accuracy on benchmark dataset")

print(f"\n✨ This analysis demonstrates proficiency in:")
print(f"   • Data preprocessing and feature engineering")
print(f"   • Model selection and hyperparameter tuning")
print(f"   • Cross-validation and performance evaluation")
print(f"   • Model interpretation and business insights")
print(f"   • Professional ML workflow and documentation")

print(f"\n🎯 Ready for deployment and real-world application!")

# Save model for future use (optional)
import joblib

model_filename = f'titanic_best_model_{best_model_name.lower().replace(" ", "_").replace("(", "").replace(")", "")}.pkl'
joblib.dump(model_for_eval, model_filename)
print(f"\n💾 Model saved as: {model_filename}")

# Save preprocessing pipeline
preprocessing_pipeline = {
    'scaler': scaler,
    'feature_columns': X_train_scaled.columns.tolist(),
    'categorical_features': categorical_features,
    'numerical_features': numerical_features
}

pipeline_filename = 'titanic_preprocessing_pipeline.pkl'
joblib.dump(preprocessing_pipeline, pipeline_filename)
print(f"💾 Preprocessing pipeline saved as: {pipeline_filename}")

print(f"\n🎊 MACHINE LEARNING ANALYSIS COMPLETE! 🎊")\n🔄 Tuning {model_name}...")
        
        # Get base model
        if model_name == 'Random Forest':
            base_model = RandomForestClassifier(random_state=42)
        elif model_name == 'Gradient Boosting':
            base_model = GradientBoostingClassifier(random_state=42)
        elif model_name == 'Logistic Regression':
            base_model = LogisticRegression(random_state=42, max_iter=1000)
        elif model_name == 'SVM':
            base_model = SVC(random_state=42, probability=True)
        
        # Perform grid search
        grid_search = GridSearchCV(
            base_model,
            param_grids[model_name],
            cv=3,  # Reduced for faster execution
            scoring='accuracy',
            n_jobs=-1,
            verbose=0
        )
        
        grid_search.fit(X_train_final, y_train_final)
        
        # Store results
        tuned_models[model_name] = grid_search.best_estimator_
        
        # Evaluate tuned model
        y_val_pred_tuned = grid_search.best_estimator_.predict(X_val_final)
        tuned_accuracy = accuracy_score(y_val_final, y_val_pred_tuned)
        
        tuning_results.append({
            'Model': model_name,
            'Best Score (CV)': grid_search.best_score_,
            'Validation Accuracy': tuned_accuracy,
            'Best Parameters': grid_search.best_params_
        })
        
        print(f"